# Simple backpropagation in Devito

In this example, we'll implement a simple convolutional neural network (CNN) in Devito, run a forward pass through it and then use backpropagation to obtain gradients necessary for training.

The CNN will have the following structure:
1. Max pool layer: input size 1x2x4x4, kernel size 2x2, stride 1x1
2. Convolutional layer: input size 1x2x3x3, kernel size 2x2x2, stride 1x1, activation ReLU
3. Flattening layer
4. Fully connected layer: input size 8x1, kernel size 3x8, activation softmax

*Size glossary: batch size x channels x height x width **or** output channels x height x width **or** height x width. Height and width are equivalent to rows and columns.*

All parameters for the forward pass will be (pseudo)random numbers generated by `np.random.rand`. Therefore, different results will be obtained each time the notebook is executed.

In [1]:
import devito.ml
import numpy as np
from sympy.functions import sign
from devito import Operator, Eq, Inc
from devito.ml.activation import ReLU

In [2]:
def backward_pass(conv_kernel, conv_bias, fc_weights, fc_bias, input_data, expected_results):
    layer1 = devito.ml.MaxPooling(kernel_size=(2, 2),
                                  input_size=(1, 2, 4, 4),
                                  generate_code=False)
    layer2 = devito.ml.Conv(kernel_size=(2, 2, 2),
                            input_size=(1, 2, 3, 3),
                            activation=ReLU(),
                            generate_code=False)
    layer_flat = devito.ml.Flat(input_size=(1, 2, 2, 2),
                                generate_code=False)
    layer3 = devito.ml.FullyConnectedSoftmax(weight_size=(3, 8),
                                             input_size=(8, 1),
                                             generate_code=False)
    
    eqs = layer1.equations() + layer2.equations(layer1.result) + \
            layer_flat.equations(layer2.result) + layer3.equations(layer_flat.result)
    
    op = Operator(eqs)
    
    layer2.kernel.data[:] = conv_kernel
    layer2.bias.data[:] = conv_bias
    
    layer3.kernel.data[:] = fc_weights
    layer3.bias.data[:] = fc_bias
    
    layer1.input.data[:] = input_data
    
    op.apply()
    
    gradients = []
    
    for i in range(3):
        result = layer3.result.data[i]
        if i == expected_results[0]:
            result -= 1
        gradients.append(result)
    
    layer3.result_gradients.data[:] = gradients
    
    dims = [layer3.bias_gradients.dimensions[0],
            layer3.kernel_gradients.dimensions[0],
            layer3.kernel_gradients.dimensions[1],
            layer_flat.result_gradients.dimensions[0],
            layer3.kernel.dimensions[1],
            layer3.result_gradients.dimensions[0],
            layer2.result_gradients.dimensions[0],
            layer2.result_gradients.dimensions[1],
            layer2.result_gradients.dimensions[2],
            layer2.kernel_gradients.dimensions[0],
            layer2.kernel_gradients.dimensions[1],
            layer2.kernel_gradients.dimensions[2],
            layer2.kernel_gradients.dimensions[3],
            layer2.bias_gradients.dimensions[0]]
    
    _, _, layer2_height, layer2_width = layer2.kernel.shape
    
    backprop_eqs = [
        Eq(layer3.bias_gradients[dims[0]], layer3.result_gradients[dims[0]]),
        Eq(layer3.kernel_gradients[dims[1], dims[2]],
           layer_flat.result[dims[2], 0] * layer3.result_gradients[dims[1]]),
        Inc(layer_flat.result_gradients[dims[4]],
            layer3.kernel[dims[5], dims[4]] * layer3.result_gradients[dims[5]]),
        Eq(layer_flat.result_gradients[dims[3]],
           layer_flat.result_gradients[dims[3]] * sign(layer_flat.result[dims[3], 0])),
        Eq(layer2.result_gradients[dims[6], dims[7], dims[8]],
           layer_flat.result_gradients[dims[6] * layer2_height * layer2_width + dims[7] * layer2_height + dims[8]]),
        Inc(layer2.bias_gradients[dims[13]], layer2.result_gradients[dims[13], dims[7], dims[8]]),
        Eq(layer2.kernel_gradients[dims[9], dims[10], dims[11], dims[12]],
            sum([layer2.result_gradients[dims[9], x, y] * layer1.result[0, dims[10], dims[11] + x, dims[12] + y]
                 for x in range(layer2.result_gradients.shape[1])
                 for y in range(layer2.result_gradients.shape[2])]))
    ]
    
    backprop_op = Operator(backprop_eqs)
    backprop_op.apply()
    
    return (layer2.kernel_gradients.data, layer2.bias_gradients.data,
            layer3.kernel_gradients.data, layer3.bias_gradients.data)

In [3]:
conv_kernel = np.random.rand(2, 2, 2)
conv_bias = np.random.rand(2)

fc_weights = np.random.rand(3, 8)
fc_bias = np.random.rand(3)

In [4]:
input_data = np.array([[[[1, 2, 3, 4],
                         [5, 6, 7, 8],
                         [9, 10, 11, 12],
                         [13, 14, 15, 16]],
                        [[-1, -2, 0, 1],
                         [-2, -3, 1, 2],
                         [3, 4, 2, -1],
                         [-2, -3, -4, 9]]]],
                     dtype=np.float64)
expected = np.array([2])

In [5]:
conv_kernel_grad, conv_bias_grad, fc_kernel_grad, fc_bias_grad = backward_pass(conv_kernel, conv_bias, fc_weights, fc_bias, input_data, expected)

/home/maksymilian/Desktop/UROP/devito/devito/types/grid.py:206: RuntimeWarning: divide by zero encountered in true_divide
  spacing = (np.array(self.extent) / (np.array(self.shape) - 1)).astype(self.dtype)
Operator `Kernel` run in 0.01 s
Operator `Kernel` run in 0.01 s


Here are kernel gradients of the convolutional layer:

In [6]:
print(conv_kernel_grad)

[[[[ 3.04044575e-05  2.90224964e-05]
   [ 2.48766131e-05  2.34946520e-05]]

  [[ 4.13694087e-05  2.12969676e-05]
   [-5.52784443e-06  1.93613486e-05]]]


 [[[-6.66596437e-05 -7.06103813e-05]
   [-8.24625944e-05 -8.64133320e-05]]

  [[-3.81183194e-05 -1.71252923e-05]
   [-1.58029507e-05 -5.76363361e-05]]]]


Here are bias gradients of the convolutional layer:

In [7]:
print(conv_bias_grad)

[-1.38196111e-06 -3.95073767e-06]


Here are kernel gradients of the fully connected layer:

In [8]:
print(fc_kernel_grad)

[[ 2.26879304e-04  2.55871633e-04  3.64882635e-04  3.98305128e-04
   2.28030656e-04  2.57022985e-04  3.66033987e-04  3.99456480e-04]
 [ 3.64237093e-06  4.10782024e-06  5.85790716e-06  6.39447932e-06
   3.66085499e-06  4.12630430e-06  5.87639122e-06  6.41296338e-06]
 [-2.30521675e-04 -2.59979454e-04 -3.70740542e-04 -4.04699608e-04
  -2.31691511e-04 -2.61149290e-04 -3.71910378e-04 -4.05869443e-04]]


Here are bias gradients of the fully connected layer:

In [9]:
print(fc_bias_grad)

[ 1.06513740e-05  1.70999533e-07 -1.08223736e-05]


## Comparison with PyTorch
To check correctness, we will carry out the same activities using PyTorch.

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Conv2d(2, 2, 2)
        self.fc = nn.Linear(8, 3)
        
    def forward(self, x):
        x = F.max_pool2d(x, 2, stride=(1, 1))
        x = F.relu(self.conv(x))
        x = x.view(-1, self.num_flat_features(x))
        x = self.fc(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [12]:
net = Net()
net.double()

with torch.no_grad():
    net.conv.weight[:] = torch.from_numpy(conv_kernel)
    net.conv.bias[:] = torch.from_numpy(conv_bias)
    
    net.fc.weight[:] = torch.from_numpy(fc_weights)
    net.fc.bias[:] = torch.from_numpy(fc_bias)

In [13]:
criterion = nn.CrossEntropyLoss()
input_data_tensor = torch.from_numpy(input_data)
outputs = net(input_data_tensor)
net.zero_grad()
loss = criterion(outputs, torch.from_numpy(expected))
loss.backward()

Here is the relative convolutional layer kernel error along with the maximum:

In [14]:
pytorch_conv_kernel_grad = net.conv.weight.grad.numpy()
conv_kernel_error = abs(conv_kernel_grad - pytorch_conv_kernel_grad) / abs(pytorch_conv_kernel_grad)
print(conv_kernel_error)
print(np.amax(conv_kernel_error))

[[[[1.93897533e-13 2.25077747e-13]
   [3.40493678e-13 3.88209657e-13]]

  [[4.19325180e-14 8.24085523e-14]
   [4.66432861e-13 1.67294854e-13]]]


 [[[7.23781197e-14 7.69655012e-14]
   [8.84189937e-14 9.17477454e-14]]

  [[2.23989206e-14 7.32021818e-14]
   [1.56939835e-13 4.18546701e-14]]]]
4.664328608794458e-13


Here is the relative convolutional layer bias error along with the maximum:

In [15]:
pytorch_conv_bias_grad = net.conv.bias.grad.numpy()
conv_bias_error = abs(conv_bias_grad - pytorch_conv_bias_grad) / abs(pytorch_conv_bias_grad)
print(conv_bias_error)
print(np.amax(conv_bias_error))

[4.65819940e-13 1.56939835e-13]
4.658199399058854e-13


Here is the relative fully connected layer kernel error along with the maximum:

In [16]:
pytorch_fc_kernel_grad = net.fc.weight.grad.numpy()
fc_kernel_error = abs(fc_kernel_grad - pytorch_fc_kernel_grad) / abs(pytorch_fc_kernel_grad)
print(fc_kernel_error)
print(np.amax(fc_kernel_error))

[[2.78362880e-14 2.79661104e-14 2.79309001e-14 2.79009018e-14
  2.78146054e-14 2.78408343e-14 2.78430441e-14 2.78204831e-14]
 [2.79059864e-14 2.80432137e-14 2.77625304e-14 2.78172639e-14
  2.77650860e-14 2.77123158e-14 2.79634874e-14 2.78691677e-14]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]]
2.8043213667842844e-14


Here is the relative fully connected layer bias error along with the maximum:

In [17]:
pytorch_fc_bias_grad = net.fc.bias.grad.numpy()
fc_bias_error = abs(fc_bias_grad - pytorch_fc_bias_grad) / abs(pytorch_fc_bias_grad)
print(fc_bias_error)
print(np.amax(fc_bias_error))

[2.78331726e-14 2.78630020e-14 0.00000000e+00]
2.7863001967068124e-14


Here is the maximum overall error:

In [18]:
print(max(np.amax(conv_kernel_error), np.amax(conv_bias_error), np.amax(fc_kernel_error), np.amax(fc_bias_error)))

4.664328608794458e-13
